In [1]:
%load_ext autoreload
%autoreload 3
%pylab widget

Populating the interactive namespace from numpy and matplotlib


In [76]:
from varian_models import *
from scipy.optimize import minimize
bcca_smaller = np.load('/home/xcite/MV_CBCT/log_kv_phantom_over_air_776x776.npy')[15:-15]

# Looking at the one slice

There is still a fudge factor of 1.03 on the projection to account for fastcats 0.97 I guess.

Also a fundge factor for the dose right now.

And Catphan_Teflon unrevised whatever that is

In [77]:
angles = np.linspace(np.pi/2,np.pi*2,2)      
kv_catphan = Catphan_404_kV()
kv_catphan.get_proj(angles,fudge=0.18)

(19, 19, 19) 19
bowtie is off False
Using the bowtie filter True
really doing it!
bowtie scatter
ratio is 70.18600063761603 number of photons 0.7694951410828342


In [78]:
kv_catphan.plot_one_proj(bcca_smaller[445,195:225])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Now lets reconstruct the whole thing

In [79]:
projs_bcca = np.zeros([887,64,512])
projs_bcca[:,17:-17,:] =  bcca_smaller[:,195:225,:]

kv_catphan.proj[:,:17,:] = 0 # Matching the images
kv_catphan.proj[:,-17:,:] = 0

angles = np.load('/home/xcite/MV_CBCT/angles_kv.npy')

In [6]:
CNR_fc = []
CNR_exp = []

im_fc = []
im_exp = []

fudge_2 = 1 # 0.97 
fudge = 0.15 #*3

ii = 4
angles_temp = angles[::ii]
kv_catphan.geomet.nVoxel = np.array([10,512,512])
kv_catphan.geomet.dVoxel = kv_catphan.geomet.sVoxel/kv_catphan.geomet.nVoxel
kv_catphan.get_proj(angles_temp,fudge=fudge) 
kv_catphan.proj[:,:17,:] = 0 # Matching the images
kv_catphan.proj[:,-17:,:] = 0
kv_catphan.geomet.nVoxel = np.array([128,512,512])
kv_catphan.geomet.dVoxel = kv_catphan.geomet.sVoxel/kv_catphan.geomet.nVoxel
kv_catphan.reconstruct('FDK',filt='ram_lak')
im_fc.append(kv_catphan.img[64])
vol_fc = kv_catphan.img
contrast_fc = kv_catphan.return_CNR(True)
CNR_fc.append(kv_catphan.return_CNR())
kv_catphan.proj = 10*projs_bcca[::ii].astype(np.float32)*fudge_2
kv_catphan.angles = angles[::ii][-1::-1]
kv_catphan.reconstruct('FDK',filt='ram_lak')
#     kv_catphan.img = np.rot90(kv_catphan.img,-1)
im_exp.append(np.rot90(kv_catphan.img[64],-1))
vol_exp = kv_catphan.img
contrast_bcca = kv_catphan.return_CNR_exp(True)
CNR_exp.append(kv_catphan.return_CNR_exp())

bowtie is off False
Using the bowtie filter True
really doing it!
bowtie scatter
ratio is 58.48845968860691 number of photons 0.7694951410828342


In [135]:
plt.close('all')

In [136]:
CNR_exp2 = []
CNR_fc2 = []
contrast_exp2 = []
contrast_fc2 = []

plt.figure()
for ii in range(60,68):
    
    CNR_exp2.append(return_CNR_exp2(vol_exp[ii]))
    contrast_exp2.append(return_CNR_exp2(vol_exp[ii],True))
    CNR_fc2.append(return_CNR_fc2(vol_fc[ii]))
    contrast_fc2.append(return_CNR_fc2(vol_fc[ii],True))
    plt.plot(ii,np.mean(CNR_exp2[-1]),'r*')
    plt.plot(ii,np.mean(CNR_fc2[-1]),'b*')
    
plt.figure()
plt.plot(np.mean(CNR_exp2,0),'r*')
plt.plot(np.mean(CNR_fc2,0),'b*')
plt.figure()
plt.plot(np.mean(contrast_exp2,0),'r*')
plt.plot(np.mean(contrast_fc2,0),'b*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [137]:
contrast_bcca = np.mean(contrast_exp2,0)
contrast_fc = np.mean(contrast_fc2,0)

def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

reses = []
rem = []

for off in np.arange(-0.2,0.2,0.01):
    for off2 in np.arange(-0.2,0.2,0.01):
        reses.append(minimize(HU_fun,[off,contrast_fc[-3]+off2],args=(contrast_fc,contrast_bcca)))
        rem.append(reses[-1].fun)

res = reses[np.argmin(rem)]

fc_data = (contrast_fc - res.x[0])*(res.x[1])
# fc_image = (im_fc[-1] - res.x[0])*(res.x[1])

# kv_catphan.img[64] = fc_image
x = [.853,0.945,.998,1.147,1.36,1.87]

plt.figure()
plt.plot(x,contrast_bcca,'s',c='darkorange',label='')
plt.plot(x,fc_data,'*',c='cornflowerblue',label='fastCAT')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [138]:
vol_fc_adj = (vol_fc - res.x[0])*(res.x[1])

In [139]:
CNR_exp3 = []
CNR_fc3 = []
contrast_exp3 = []
contrast_fc3 = []

plt.figure()
for ii in range(60,68):
    
    CNR_exp3.append(return_CNR_exp2(vol_exp[ii]))
    contrast_exp3.append(return_CNR_exp2(vol_exp[ii],True))
    CNR_fc3.append(return_CNR_fc2(vol_fc_adj[ii]))
    contrast_fc3.append(return_CNR_fc2(vol_fc_adj[ii],True))
    plt.plot(ii,np.mean(CNR_exp3[-1]),'r*')
    plt.plot(ii,np.mean(CNR_fc3[-1]),'b*')
    
plt.figure()
plt.plot(np.mean(CNR_exp3,0),'r*')
plt.plot(np.mean(CNR_fc3,0),'b*')
plt.figure()
plt.plot(np.mean(contrast_exp3,0),'r*')
plt.plot(np.mean(contrast_fc3,0),'b*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Now lets look at all of them at once

In [84]:
CNR_fc = []
CNR_exp = []

im_fc = []
im_exp = []

fudge_2 = 1 # 0.97 
fudge = 0.146*1.1 #*3

CNR_result_fc = []
CNR_result_exp = []

np.random.seed(100)

def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

for ii in range(4,3,-1):
    angles_temp = angles[::ii]
    kv_catphan.geomet.nVoxel = np.array([10,512,512])
    kv_catphan.geomet.dVoxel = kv_catphan.geomet.sVoxel/kv_catphan.geomet.nVoxel
    kv_catphan.get_proj(angles_temp,fudge=fudge) 
    kv_catphan.proj[:,:17,:] = 0 # Matching the images
    kv_catphan.proj[:,-17:,:] = 0
    kv_catphan.geomet.nVoxel = np.array([128,512,512])
    kv_catphan.geomet.dVoxel = kv_catphan.geomet.sVoxel/kv_catphan.geomet.nVoxel
    kv_catphan.reconstruct('FDK',filt='ram_lak')
    vol_fc = kv_catphan.img
    
    kv_catphan.proj = 10*projs_bcca[::ii].astype(np.float32)*fudge_2
    kv_catphan.angles = angles[::ii][-1::-1]
    kv_catphan.reconstruct('FDK',filt='ram_lak')
    vol_exp = kv_catphan.img

    contrast_exp2 = []
    contrast_fc2 = []

    for ii in range(60,68):

        contrast_exp2.append(return_CNR_exp2(vol_exp[ii],True))
        contrast_fc2.append(return_CNR_fc2(vol_fc[ii],True))
    
    contrast_bcca = np.mean(contrast_exp2,0)
    contrast_fc = np.mean(contrast_fc2,0)

    reses = []
    rem = []

    for off in np.arange(-0.2,0.2,0.01):
        for off2 in np.arange(-0.2,0.2,0.01):
            reses.append(minimize(HU_fun,[off,contrast_fc[-3]+off2],args=(contrast_fc,contrast_bcca)))
            rem.append(reses[-1].fun)

    res = reses[np.argmin(rem)]

    fc_data = (contrast_fc - res.x[0])*(res.x[1])
    vol_fc_adj = (vol_fc - res.x[0])*(res.x[1])
    
    CNR_exp4 = []
    CNR_fc4 = []

    for ii in range(60,68):

        CNR_exp4.append(return_CNR_exp2(vol_exp[ii]))
        CNR_fc4.append(return_CNR_fc2(vol_fc_adj[ii]))
    
    CNR_result_fc.append((np.mean(CNR_fc4),np.std(np.mean(CNR_fc4,1))))
    CNR_result_exp.append((np.mean(CNR_exp4),np.std(np.mean(CNR_exp4,1))))

bowtie is off False
Using the bowtie filter True
really doing it!
bowtie scatter
ratio is 62.62164417326847 number of photons 0.7694951410828342


In [90]:
np.save('CNR_kv_exp',np.mean(CNR_exp4,0))
np.save('CNR_kv_fc',np.mean(CNR_fc4,0))

x = [.853,0.945,.998,1.147,1.36,1.87]

plt.figure(figsize=[4,7])
plt.subplot(211)
plt.title('CNR Comparison')
plt.errorbar(x,np.mean(CNR_exp4,0),np.std(CNR_exp4,0),fmt='s',color='darkorange',label='')
plt.errorbar(np.array(x)+0.02,np.mean(CNR_fc4,0),np.std(CNR_fc4,0),fmt='*',color='cornflowerblue',label='fastCAT')
plt.grid()
plt.ylabel('CNR')
# plt.xlabel('Relative Electron Density')
plt.subplot(212)
plt.errorbar(x,np.mean(CNR_exp3,0),np.std(CNR_exp3,0),fmt='s',color='darkorange',label='')
plt.errorbar(np.array(x)+0.02,np.mean(CNR_fc3,0),np.std(CNR_fc3,0),fmt='*',color='cornflowerblue',label='fastCAT')
plt.grid()
plt.ylabel('CNR')
plt.xlabel('Relative Electron Density')
plt.tight_layout()
plt.savefig('Comparison_CNR.png',dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [176]:
CNR_err_fc = np.std(CNR_fc3)
CNR_err_exp = np.std(CNR_exp3)

plt.figure()
plt.errorbar([1,2,3,4],[CNR_result_fc[ii][0] for ii in range(4)],fmt='b*',yerr =  [CNR_result_fc[ii][1] for ii in range(4)])
plt.errorbar([1.1,2.1,3.1,4.1],[CNR_result_exp[ii][0] for ii in range(4)],fmt='r*',yerr = [CNR_result_exp[ii][1] for ii in range(4)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [177]:
np.save('CNR_fc_by_dose',[CNR_result_fc[ii][0] for ii in range(4)])
np.save('CNR_err_fc_by_dose',[CNR_result_fc[ii][1] for ii in range(4)])
np.save('CNR_exp_by_dose',[CNR_result_exp[ii][0] for ii in range(4)])
np.save('CNR_err_exp_by_dose',[CNR_result_exp[ii][1] for ii in range(4)])

# This is the one for the gos

In [46]:
from glob import glob

In [59]:
CNR_fc = []
CNR_exp = []

im_fc = []
im_exp = []

fudge_2 = 1 # 0.97 
fudge = 0.146/1.08*1.22*1.5#*4.5

CNR_result_fc = []
CNR_result_exp = []

angles = np.load('/home/xcite/MV_CBCT/angles_6x.npy')
exp_files = np.array(glob('/home/xcite/MV_CBCT/recon_6x_300_*.npy'))[np.array([1,3,0,2])]

np.random.seed(100)

def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

kv_catphan = Catphan_404_6x()

for ii in range(3,2,-1):
    angles_temp = angles[::ii]
    kv_catphan.geomet.nVoxel = np.array([10,512,512])
    kv_catphan.geomet.dVoxel = kv_catphan.geomet.sVoxel/kv_catphan.geomet.nVoxel
    kv_catphan.get_proj(angles_temp,fudge=fudge) 
    kv_catphan.geomet.nVoxel = np.array([256,512,512])
    kv_catphan.geomet.dVoxel = kv_catphan.geomet.sVoxel/kv_catphan.geomet.nVoxel
    kv_catphan.reconstruct('FDK',filt='ram_lak')
    
    vol_fc = kv_catphan.img
    vol_exp = np.rot90(np.load(exp_files[ii-1]).T,0)

    contrast_exp2 = []
    contrast_fc2 = []

    for ii in range(20):

        contrast_exp2.append(return_CNR_exp3(vol_exp[25 +ii],True))
        contrast_fc2.append(return_CNR_fc3(vol_fc[120+ii],True))
    
    contrast_bcca = np.mean(contrast_exp2,0)
    contrast_fc = np.mean(contrast_fc2,0)

    reses = []
    rem = []

    for off in np.arange(-0.2,0.2,0.01):
        for off2 in np.arange(-0.2,0.2,0.01):
            reses.append(minimize(HU_fun,[off,contrast_fc[-3]+off2],args=(contrast_fc,contrast_bcca)))
            rem.append(reses[-1].fun)

    res = reses[np.argmin(rem)]

    vol_fc_adj = (vol_fc - res.x[0])*(res.x[1])
    fc_data = (contrast_fc - res.x[0])*(res.x[1])
    
    plt.figure()
    plt.plot(fc_data,'b*')
    plt.plot(contrast_bcca,'r*')
    
    CNR_fc3 = []
    CNR_exp3 = []
    
    for ii in range(20):

        CNR_exp3.append(return_CNR_exp3(vol_exp[25 + ii]))
        CNR_fc3.append(return_CNR_fc3(vol_fc_adj[120+ii]))
    
    plt.figure()
    plt.plot(np.mean(CNR_exp3,0),'r*')
    plt.plot(np.mean(CNR_fc3,0),'b*')
    CNR_result_fc.append((np.mean(CNR_fc3),np.std(np.mean(CNR_fc3,1))))
    CNR_result_exp.append((np.mean(CNR_exp3),np.std(np.mean(CNR_exp3,1))))

(19, 41, 41) 19
bowtie is off False
Using the bowtie filter True
really doing it!
ratio is 1915.6476207485534 number of photons 0.02636808212587352


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
np.std(CNR_exp3,0)

array([0.34996606, 0.35465521, 0.69550135, 0.56201382, 0.58896832,
       0.74425133])

In [74]:
np.save('CNR_6x_exp',np.mean(CNR_exp3,0))
np.save('CNR_6x_fc',np.mean(CNR_fc3,0))

x = [.853,0.945,.998,1.147,1.36,1.87]

plt.figure()
plt.errorbar(x,np.mean(CNR_exp3,0),np.std(CNR_exp3,0),fmt='s',color='darkorange',label='')
plt.errorbar(np.array(x)+0.02,np.mean(CNR_fc3,0),np.std(CNR_fc3,0),fmt='*',color='cornflowerblue',label='fastCAT')
plt.grid()
plt.ylabel('CNR')
plt.xlabel('Relative Electron Density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Relative Electron Density')

In [19]:
nviews = 180

s = fc.Spectrum()
s.load('Varian_truebeam_phasespace')
kernel = fc.Kernel(s,'CuGOS-784-micrometer')
angles = np.linspace(0,np.pi*2,nviews)

phantom = fc.Catphan_404()
phantom.phan_map = ['air','G4_POLYSTYRENE','G4_POLYVINYL_BUTYRAL','G4_POLYVINYL_BUTYRAL','CATPHAN_Delrin','G4_POLYVINYL_BUTYRAL','CATPHAN_Teflon_revised','air','CATPHAN_PMP','G4_POLYVINYL_BUTYRAL','CATPHAN_LDPE','G4_POLYVINYL_BUTYRAL','CATPHAN_Polystyrene','air','CATPHAN_Acrylic','air','CATPHAN_Teflon_revised','air','air','air','air'] 
# phantom.phan_map = ['air','CATPHAN_Polystyrene','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Delrin','CATPHAN_Acrylic','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_Acrylic','CATPHAN_LDPE','CATPHAN_Acrylic','CATPHAN_Polystyrene','air','CATPHAN_Acrylic'] 
phantom.return_projs(kernel,s,angles,det_on=True,scat_on=False,mgy=0.0)

(19, 41, 41) 19
bowtie is off False
really doing it!
ratio is None number of photons 0.010763158668836276


In [22]:
X = np.array(phantom.proj)

plt.figure()
for ii in range(X.shape[0]):
    
    imshow(np.rot90(X[ii, :, :]),cmap='gray',vmax = 16,vmin=0,aspect='auto')        
#     plt.ylabel('projection %s' % ii)
    plt.xticks([])
    plt.yticks([])
    plt.savefig(f'pngs/{ii}.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [274]:
CNR_err_fc = np.std(CNR_fc3)
CNR_err_exp = np.std(CNR_exp3)

plt.figure()
plt.errorbar([1,2,3,4],[CNR_result_fc[ii][0] for ii in range(3,-1,-1)],fmt='b*',yerr =  [CNR_result_fc[ii][1] for ii in range(4)])
plt.errorbar([1.1,2.1,3.1,4.1],[CNR_result_exp[ii][0] for ii in range(4)],fmt='r*',yerr = [CNR_result_exp[ii][1] for ii in range(4)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [275]:
np.save('CNR_fc_by_dose_kv',[CNR_result_fc[ii][0] for ii in range(4)])
np.save('CNR_err_fc_by_dose_kv',[CNR_result_fc[ii][1] for ii in range(4)])
np.save('CNR_exp_by_dose_kv',[CNR_result_exp[ii][0] for ii in range(4)])
np.save('CNR_err_exp_by_dose_kv',[CNR_result_exp[ii][1] for ii in range(4)])

In [1]:
%pylab widget

Populating the interactive namespace from numpy and matplotlib


In [91]:
fc = np.load('CNR_fc_by_dose_kv.npy')
fc_err = np.load('CNR_err_fc_by_dose_kv.npy')
exp = np.load('CNR_exp_by_dose_kv.npy')
exp_err = np.load('CNR_err_exp_by_dose_kv.npy')

dose_lab = np.array(np.round([300,300/2,300/3,300/4],1))[-1::-1]
dose = np.array([6,4,2,0])[-1::-1]
plt.figure(figsize=[4,7])
plt.subplot(212)
plt.bar(dose,fc,yerr=fc_err,width=0.4,color='cornflowerblue')
plt.bar(dose+0.5,exp[-1::-1],yerr=exp_err,width=0.4,color='darkorange')
plt.grid('on')
plt.xticks(ticks=dose,labels= dose_lab)
plt.xlabel('MU')
plt.ylabel('CNR')
plt.legend(['Fastcat 6 MV','Varian 6 MV'])
plt.ylim([0,14])

fc = np.load('CNR_fc_by_dose.npy')
fc_err = np.load('CNR_err_fc_by_dose.npy')
exp = np.load('CNR_exp_by_dose.npy')
exp_err = np.load('CNR_err_exp_by_dose.npy')

dose_lab = np.array(np.round([21.1,21.1/2,21.1/3,21.1/4],1))[-1::-1]
plt.subplot(211)
plt.title('CNR vs Dose')
plt.bar(dose,fc,yerr=fc_err,width=0.4,color='cornflowerblue')
plt.bar(dose+0.5,exp,yerr=exp_err,width=0.4,color='darkorange')
plt.grid('on')
plt.xticks(ticks=dose,labels= dose_lab)
plt.xlabel('CTDI$_{vol}$')
plt.ylabel('CNR')
plt.legend(['Fastcat 100 kVp','Varian OBI 100 kVp'])
plt.ylim([0,20])
plt.tight_layout()
plt.savefig('Comparison_dose_CNR.png',dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [82]:
def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

from scipy.optimize import minimize

reses = []
rem = []

for off in np.arange(-0.2,0.2,0.01):
    reses.append(minimize(HU_fun,[off,contrast_fc[-3]+0.12],args=(contrast_fc,contrast_bcca)))
    rem.append(reses[-1].fun)

res = reses[np.argmin(rem)]

fc_data = (contrast_fc - res.x[0])*(res.x[1])
fc_image = (im_fc[-1] - res.x[0])*(res.x[1])

kv_catphan.img[64] = fc_image
x = [.853,0.945,.998,1.147,1.36,1.87]

plt.figure()
plt.plot(x,contrast_bcca,'s',c='darkorange',label='')
plt.plot(x,fc_data,'*',c='cornflowerblue',label='fastCAT')
# plt.figure()
# plt.plot(CNR_exp[0],'r*')
# plt.plot(kv_catphan.return_CNR(),'b*')
# new_CNR_fc.append(return_CNR(fc_image,im)[1])    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [127]:
np.save('fastcat_images_kv_worst-best',im_fc)
np.save('exp_images_kv_worst-best',im_exp)
np.save('CNR_fc_kv',CNR_fc_kv)
np.save('CNR_exp_kv',CNR_exp)
np.save('vol_fc_kv_adj',vol_fc_adj)
np.save('vol_exp_kv',vol_exp)

In [374]:
plt.imsave('im_for_imagej.png',np.hstack((im_fc[0],im_exp[0])),cmap='gray')
!imagej im_for_imagej.png

Open other images in this ImageJ panel as follows:
  imagej -p 1 <image1> [<image2> ... <imageN>]

Fontconfig error: "/home/xcite/.config/fontconfig/fonts.conf", line 5: no element found
Fontconfig error: "/home/xcite/.config/fontconfig/fonts.conf", line 5: invalid attribute 'target'


In [44]:
CNR_fc = np.load('CNR_fc_kv.npy')
CNR_exp = np.load('CNR_exp_kv.npy')

In [378]:
plt.imsave('im_for_imagej.png',np.hstack((fc_image,recon_slice)),cmap='gray')
!imagej im_for_imagej.png

Open other images in this ImageJ panel as follows:
  imagej -p 1 <image1> [<image2> ... <imageN>]

Fontconfig error: "/home/xcite/.config/fontconfig/fonts.conf", line 5: no element found
Fontconfig error: "/home/xcite/.config/fontconfig/fonts.conf", line 5: invalid attribute 'target'


In [379]:
from scipy.optimize import minimize
CNR_fc = np.load('CNR_fc_kv.npy')
CNR_exp = np.load('CNR_exp_kv.npy')

im_fc = np.load('fastcat_images_kv_worst-best.npy')
im_exp = np.load('exp_images_kv_worst-best.npy')

new_CNR_fc = []

def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

def return_CNR_fc(proj,return_contrast = False):

    im = create_mask(proj.shape,r=5.75,radius=0.4)
    contrast_fc, CNR_fc, noise_fc = return_CNR(proj,im)
#         print(contrast_fc ,'fastcat')
    if return_contrast:
        return contrast_fc
    else:
        return CNR_fc

def return_CNR_exp(proj,return_contrast = False):

    recon_slice = (proj)
    im = create_mask(recon_slice.shape,r=5.85,off = [-0.15,-0.02],rot = 0.7,radius=0.40)
    contrast_fc, CNR_fc, noise_fc = return_CNR(recon_slice,im)
#         print(contrast_fc, 'exp')
    if return_contrast:
        return contrast_fc
    else:
        return CNR_fc

im = create_mask(im_fc.shape,r=5.75,radius=0.4)

for ii in range(3,-1,-1):
    
    contrast_fc = return_CNR_fc(im_fc[ii],True) 
    contrast_bcca = return_CNR_exp(im_exp[ii],True) 
    
    if ii == 3:
        reses = []
        rem = []

        for off in np.arange(-0.2,0.2,0.01):
            reses.append(minimize(HU_fun,[off,contrast_fc[-3]-0.01],args=(contrast_fc,contrast_bcca)))
            rem.append(reses[-1].fun)

        res = reses[np.argmin(rem)]

    fc_data = (contrast_fc - res.x[0])*(res.x[1])
    fc_image = (im_fc[ii] - res.x[0])*(res.x[1])
    
    new_CNR_fc.append(return_CNR(fc_image,im)[1])
    
    x = [.853,0.945,.998,1.147,1.36,1.87]

    plt.figure(figsize=[8,8])
    plt.subplot(221)
    plt.plot(x,contrast_bcca,'s',c='darkorange',label='')
    plt.plot(x,fc_data,'*',c='cornflowerblue',label='fastCAT')
    plt.legend()
    plt.subplot(222)
    plt.plot(recon_slice[256,:],c='darkorange')
    plt.plot(np.rot90(fc_image,0)[256,:],c='cornflowerblue')
    plt.subplot(223)
    plt.imshow(np.rot90(fc_image,0),cmap='gray',vmin=0.13,vmax=0.35)
    plt.subplot(224)
    plt.imshow(np.rot90(recon_slice,0),cmap='gray',vmin=0.13,vmax=0.35)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [380]:
plt.figure()
plt.plot((new_CNR_fc[3]),'b*')
plt.plot(CNR_exp[3],'r*')
# plt.figure()
# plt.plot(contrast_fc,'b*')
# plt.plot(contrast_bcca,'r*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [218]:
res

      fun: 0.0026893317699432373
 hess_inv: array([[0.01104779, 0.01104779],
       [0.01104779, 0.14830004]])
      jac: array([0., 0.])
  message: 'Optimization terminated successfully.'
     nfev: 81
      nit: 4
     njev: 27
   status: 0
  success: True
        x: array([-0.03572898,  0.82787716])

In [197]:
plt.figure()
plt.plot(CNR_fc[3],'b*')
plt.plot(CNR_exp[3],'r*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [172]:
plt.figure()
plt.imshow(np.rot90(im_exp[0],-1))
# plt.imshow(im)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [196]:
plt.figure()
for ii in range(4):
    plt.plot(ii,np.mean(CNR_fc[ii],0),'r*')
    plt.plot(ii,np.mean(CNR_exp[ii],0),'b*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [121]:
recon_slice = np.rot90(kv_catphan.img[64],3)
im = create_mask(recon_slice.shape,r=5.85,radius=0.45,off = [-0.15,-0.02],rot = 0.7)
contrast_exp, CNR_exp, noise_fc = return_CNR(recon_slice,im,show_map=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [122]:
print(contrast_exp)

[0.16167541 0.17474125 0.18429291 0.20837197 0.24231511 0.3306908 ]


In [ ]:
# bcca_recon = np.load('/home/xcite/MV_CBCT/recon_kV_full.npy').T #recon_kv_jan_2021.npy')
# recon_slice = np.rot90(np.mean(bcca_recon.T[39:40],0)).copy()
# im = create_mask(recon_slice.shape,r=5.8,off = [0.2,0.05],rot = 1)
# contrast_bcca, CNR_bcca, noise_bcca = return_CNR(recon_slice,im)#,show_map=True)

def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

reses = []
rem = []

for off in np.arange(-0.2,0.2,0.01):
    reses.append(minimize(HU_fun,[contrast_fc[2],contrast_fc[-3]+off],args=(contrast_fc,contrast_bcca)))
    rem.append(reses[-1].fun)
    
res = reses[np.argmin(rem)]
    
fc_data = (contrast_fc - res.x[0])*(res.x[1])
fc_image = (phantom.img[4] - res.x[0])*(res.x[1])

x = [.853,0.945,.998,1.147,1.36,1.87]

plt.figure(figsize=[8,8])
plt.subplot(221)
plt.plot(x,contrast_bcca,'s',c='darkorange',label='')
plt.plot(x,fc_data,'*',c='cornflowerblue',label='fastCAT')
plt.legend()
plt.subplot(222)
plt.plot(recon_slice[256,:],c='darkorange')
plt.plot(np.rot90(fc_image,0)[256,:],c='cornflowerblue')
plt.subplot(223)
plt.imshow(np.rot90(fc_image,0),cmap='gray',vmin=0.013,vmax=0.027)
plt.subplot(224)
plt.imshow(np.rot90(recon_slice,2),cmap='gray',vmin=0.013,vmax=0.027)